## Initializing opencv haarcascade face detection network
https://github.com/opencv/opencv/tree/master/data/haarcascades
explained in: https://www.youtube.com/watch?v=7IFhsbfby9s&t=300s (or gitHub)

In [17]:
import cv2
import os

In [18]:
def convertImg(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def faceNetLocalize(img, **kwargs):
    scaleFactor = kwargs.get('scaleFactor', 1.1) #between 1.05 (quality) and 1.4 (speed) recommended (scale of the faces we search for)
    minNeighbors = kwargs.get('minNeighbors', 4) #between 3 (quantity) and 6 (quality) recommended
    minSize = kwargs.get('minSize', (10, 10)) #min size of a face in the picture
    faceNet = kwargs.get('faceNet', init_faceNet())
    
    img_cvt = convertImg(img)
    return faceNet.detectMultiScale3(img_cvt, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize, outputRejectLevels = True)

def init_faceNet(**kwargs):
    path = kwargs.get('path', 'haarcascade_frontalface_default.xml')
    return cv2.CascadeClassifier(path)


In [20]:
#TODO: testing with different model types
#for example: eye model + larger bounding box towards the bottom

## Mask classifier

foundation: https://www.tensorflow.org/tutorials/load_data/images

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPool2D, Conv2D, Input, Dense, Flatten, AveragePooling2D, Dropout
import tensorflow.keras.layers as lays
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomContrast, RandomFlip, RandomRotation
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential, losses as lfs
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from tensorflow.keras.callbacks import ModelCheckpoint

#### Data augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

In [23]:
augmentation = Sequential([
  RandomFlip("horizontal"),
  RandomRotation(0.4),
  RandomContrast(0.5)
])

##### variables:

In [24]:
batch_size = 32
img_size = (180, 180)
img_size_vgg = (224, 224)
epochs = 11
checkpoint_path = "mask_model/weights.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
imgs_path = os.path.join('..', 'img')
num_classes = 2

correct_usage=  'correct usage: \n' + 'predict([path to image], \'category\' \n' + 'predict([path to image], \'probabilities\' \n' + 'predict([path to image], \'detection\' \n' + 'predict(\'live_detection\')'

##### loading dataset:

In [25]:
def load_dataset(**kwargs):
    imgs_path = kwargs.get('imgs_path', os.path.join('..', 'img'))
    img_size = kwargs.get('img_size', (180, 180))
    batch_size = kwargs.get('batch_size', 32)

    train_ds = image_dataset_from_directory(imgs_path,  validation_split=0.2, subset="training",  seed=3, image_size=img_size,  batch_size=batch_size)
    val_ds = image_dataset_from_directory(imgs_path,  validation_split=0.2, subset="validation",  seed=3, image_size=img_size,  batch_size=batch_size)
    labels = train_ds.class_names

    y_test = np.concatenate([y for _, y in val_ds], axis=0)
    x_test = np.concatenate([x for x, _ in val_ds], axis=0)
    return train_ds, val_ds, labels, y_test, x_test

Found 293 files belonging to 2 classes.
Using 235 files for training.
Found 293 files belonging to 2 classes.
Using 58 files for validation.


### Model selection & Training

In [95]:
#Hypermodels
#https://www.analyticsvidhya.com/blog/2021/06/create-convolutional-neural-network-model-and-optimize-using-keras-tuner-deep-learning/
#https://www.tensorflow.org/tutorials/keras/keras_tuner

def basic_model_builder(hp):
  
    basic_model = Sequential([
        Rescaling(1. /255),
        augmentation,
        Conv2D(filters=hp.Int('c1_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c1_kernel', values=[3,5]), activation='relu'),
        AveragePooling2D(pool_size=(7,7)),
        Flatten(name="flatten"),
        Dense(units=hp.Int('d1_units', min_value=32, max_value=512, step=32), activation="relu"),
        Dropout(0.5),#drops small confidences
        Dense(num_classes, activation="softmax")
    ])

    basic_model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=lfs.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return basic_model

def small_model_builder(hp):
  
    model = Sequential([
        Rescaling(1. /255),
        augmentation,
        Conv2D(filters=hp.Int('c1_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c1_kernel', values=[3,5]), activation='relu'),
        Conv2D(filters=hp.Int('c2_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c2_kernel', values=[3,5]), activation='relu'),
        MaxPool2D(pool_size=(3,3)),
        Flatten(name="flatten"),
        Dense(units=hp.Int('d1_units', min_value=32, max_value=512, step=32), activation="relu"),
        Dropout(0.5),#drops small confidences
        Dense(num_classes, activation="softmax")
    ])

    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=lfs.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

def tune_model(model_builder, xs, ys):
    #tuner = kt.RandomSearch(model_builder, objective='val_accuracy', max_trials=5)
    #tuner = kt.Hyperband(model_builder, objective='accuracy', max_epochs=10, factor=3)
    tuner = kt.RandomSearch(kt.applications.HyperResNet(input_shape=(180, 180, 3), classes=2), objective='val_loss', max_trials=5)
    tuner.search(xs, ys, epochs=50, validation_split=0.2)
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 9 Complete [00h 05m 22s]
accuracy: 0.568965494632721

Best accuracy So Far: 0.6379310488700867
Total elapsed time: 00h 58m 19s

Search: Running Trial #10

Hyperparameter    |Value             |Best Value So Far 
c1_filter         |48                |128               
c1_kernel         |5                 |5                 
c2_filter         |240               |128               
c2_kernel         |5                 |5                 
d1_units          |480               |224               
learning_rate     |0.001             |0.01              
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/2


In [26]:
basic_model = Sequential([ 
    Rescaling(1. /255),
    augmentation,
    Conv2D(32, (3,3), activation='relu'),
    AveragePooling2D(pool_size=(7,7)),
    Flatten(name="flatten"),
    Dense(128, activation="relu"),
    Dropout(0.5),#drops small confidences
    Dense(num_classes, activation="softmax")
    ])

small_model = Sequential([ 
    Rescaling(1. /255),
    augmentation,
    Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
    Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
    MaxPool2D(pool_size=(3,3)),
    Flatten(name="flatten"),
    Dense(units=224, activation="relu"),
    Dropout(0.5),#drops small confidences
    Dense(num_classes, activation="softmax")
    ])

vgg_small_model = Sequential([ 
    Rescaling(1. /255),
    Conv2D(64, (3,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(128, (3,3), activation='relu'),
    Conv2D(128, (3,3), activation='relu'),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    Flatten(name="flatten"),
    Dense(256, activation="relu"),
    Dropout(0.5),#drops small confidences
    Dense(num_classes, activation="softmax")
    ])

vgg_model = Sequential([
    Rescaling(1. /255),
    Conv2D(input_shape=(224,224,3), filters=64, kernel_size=(3,3), padding="same", activation="relu", strides=(1,1)), 
    Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2)),
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2)),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2)),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2)),
    Flatten(),
    Dense(units=4096, activation="relu"),
    Dense(units=4096, activation="relu"),
    Dense(units=4, activation="softmax")
])
model = small_model


about the VGG-model: https://neurohive.io/en/popular-networks/vgg16/

In [27]:
def train_model(model, train_ds, val_ds, **kwargs):
    epochs = kwargs.get('epochs', 10)
    checkpoint_path = kwargs.get('checkpoint_path', checkpoint_path = "mask_model/weights.ckpt")

    #Callback to save model's weights
    #https://www.tensorflow.org/tutorials/keras/save_and_load
    cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose = 1)

    model.compile(optimizer=Adam(0.01), loss=lfs.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[cp_callback])
    return history

Epoch 1/11


C:\Users\Julie\anaconda3\lib\site-packages\tensorflow\python\keras\backend.py:4942: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


8/8 [==============================] - 174s 20s/step - loss: 2722.5886 - accuracy: 0.5915 - val_loss: 0.6939 - val_accuracy: 0.4483

Epoch 00001: saving model to mask_model\weights.ckpt
Epoch 2/11
8/8 [==============================] - 131s 16s/step - loss: 1.1520 - accuracy: 0.5362 - val_loss: 0.6945 - val_accuracy: 0.4483

Epoch 00002: saving model to mask_model\weights.ckpt
Epoch 3/11
8/8 [==============================] - 117s 14s/step - loss: 2.4570 - accuracy: 0.5106 - val_loss: 0.7862 - val_accuracy: 0.4655

Epoch 00003: saving model to mask_model\weights.ckpt
Epoch 4/11
8/8 [==============================] - 114s 13s/step - loss: 0.8825 - accuracy: 0.5319 - val_loss: 0.6992 - val_accuracy: 0.4483

Epoch 00004: saving model to mask_model\weights.ckpt
Epoch 5/11
8/8 [==============================] - 115s 14s/step - loss: 0.7020 - accuracy: 0.5149 - val_loss: 0.6965 - val_accuracy: 0.4483

Epoch 00005: saving model to mask_model\weights.ckpt
Epoch 6/11
8/8 [======================

#### Testing/Evaluation (mask):
https://www.tensorflow.org/guide/keras/train_and_evaluate

In [29]:
def evaluate_model(x_test, y_test, model):
    results = model.evaluate(x_test, y_test, batch_size=32)
    print(results)
    
    y_pred_confidences = model.predict(x_test)
    y_pred = [np.argmax(cs) for cs in y_pred_confidences]
    print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.55      1.00      0.71        32
           1       0.00      0.00      0.00        26

    accuracy                           0.55        58
   macro avg       0.28      0.50      0.36        58
weighted avg       0.30      0.55      0.39        58



C:\Users\Julie\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julie\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julie\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Predictions:

In [30]:
def load_model(checkpoint_path, model):
    model.load_weights(checkpoint_path)
    

In [31]:
def maskPredict(img, labels):
    pred = model.predict(img[None])
    #TODO: richtiges mapping zwischen index/label?
    label_index = np.argmax(pred)
    return labels[label_index], pred[0][label_index]


#mode can be 'category', 'probabilities', 'detection', 'live_detection'
def predict(img_path, mode):
    #load_img
    img = load_img(img_path, target_size = img_size)
    img = img_to_array(img)
    
    if mode=='detection':
        detect(img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return
    if mode=='live_detection':
        live_det()
        return
        
    if mode=='category':
        label, confidence = maskPredict(img)
        return label
        
    if mode=='probabilities':
        #TODO: schöneres Format
        return model.predict(img[None])

    else:
        print(correct_usage)

#def predict(mode):
#    if mode=='live_detection':
#        predict('', mode)
#    else:
#        print(correct_usage)

def detect(img):
    faceLocs, rejectLevels, confidences = faceNetLocalize(img)
        
    for (x, y, w, h) in faceLocs:
        #crop image and predict label of cropped image
        img_crop = img[y:y+h, x:x+w]
        label, confidence_mask = maskPredict(img)
        #show label/ bounding box on image
        cv2.putText(img, f"{label}, confidence:{confidence_mask}", (x+w-30, y+h), cv2.FONT_HERSHEY_PLAIN, 1.0, cv2.CV_RGB(0,255,0), 2.0) 
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    cv2.imshow('live_output', img)

def live_det():
    #TODO: errorhandling for camera
    
    wait_time = 10 #time in ms to wait before refreshing feed
    camera = cv2.VideoCapture(0) #Input value might differ on different systems
    
    while(True):
        _, img = camera.read()

        detect(img)

        #wait for ESC or q
        if (cv2.waitKey(wait_time) & 0xFF) in [27, ord('q')]: 
            break


    camera.release()
    return 'live_output'


In [32]:
#img_path='../img/ffp2/IMG_1596_(657, 421, 1426, 1426).png'
#img_path='../img/no_mask/IMG_1323.JPG_(1025, 427, 984, 984).png'
#img_path='../img/op_mask/IMG_1337.JPG_(878, 710, 954, 954).png'
#img_path='../img/other_mask/IMG_1327.JPG_(863, 476, 1097, 1097).png'

#predict(img_path, 'live_detection')